In [2]:
# packages included with Anaconda installation
import numpy as np
from os.path import join, exists
from os import mkdir
from tifffile import imread
import matplotlib.pyplot as plt
from itertools import product
from glob import glob
from pathlib import Path
from scipy.ndimage import binary_erosion
import ipympl
%matplotlib ipympl

# custom packages
from highpass_filter import highpass_filter

In [3]:
def calc_cumulants(image, mask):
    """
    """
    nframes, nchannels, nrows, ncols = image.shape
    N = mask.sum()
    
    image_delta = np.zeros([nframes, nchannels, nrows, ncols], dtype='float')
    image_delta_sq = np.zeros([nframes, nchannels, nrows, ncols], dtype='float')
    
    mask_shift = mask[:,:,:-1] * mask[:,:,1:]
    
    kappa1 = np.zeros(nchannels, dtype='float')
    kappa2 = np.zeros([nchannels, nchannels], dtype='float')
    mu22   = np.zeros([nchannels, nchannels], dtype='float')
    var1   = np.zeros(nchannels, dtype='float')
    var2   = np.zeros([nchannels, nchannels], dtype='float')
    
    for ch in range(nchannels):
        kappa1[ch] = image[:,ch][mask].mean()
    
    for frame in range(nframes):
        for channel in range(nchannels):
            image_delta[frame, channel] = image[frame,channel] - image[frame,channel][mask[frame]].mean()
            image_delta[frame, channel][~mask[frame]] = 0
            image_delta_sq[frame, channel] = image_delta[frame, channel]**2
    
    ch_pairs = list(product(range(nchannels), repeat=2))
    for ich, (ch1, ch2) in enumerate(ch_pairs):
        kappa2[ch1,ch2] = (image_delta[:,ch1,:,:-1] * image_delta[:,ch2,:,1:])[mask_shift].mean()
        mu22[ch1,ch2]   = (image_delta[:,ch1,:,:-1]**2 * image_delta[:,ch2,:,1:]**2)[mask_shift].mean()
        
    var1 = kappa2[range(nchannels),range(nchannels)] / N
    var2 = (kappa2**2 - 2*kappa2 + mu22) / N
        
    return kappa1, kappa2, var1, var2

In [4]:
folders = [
         r'D:\Data\FFS\210519 da gprotein stim repeat\dopamine',
         r'D:\Data\FFS\210520 epinephrine stimulation\epinephrine'
]

maskfolder = r'unmixed images\masks'
savefolder = r'cumulants - with filtering'

window_sz = 3
make_correction = True

In [5]:
for folder in folders:
    image_files = glob(join(folder, '??*.lsm'))
    if not exists(join(folder, maskfolder, savefolder)):
        mkdir(join(folder, maskfolder, savefolder))
    for image_file in image_files:
        image_num = Path(image_file).stem
        mask_files = glob(join(folder, maskfolder, image_num+'*ROI??.npy'))
        if mask_files:
            image = imread(image_file)[0]
            for mask_file in mask_files:
                mask = np.load(mask_file)
                binary_erosion(mask, 
                               structure=np.ones([window_sz,1,1], dtype='bool'), 
                               border_value=True, 
                               output=mask)
                mask_label = Path(mask_file).stem
                image_filtered = highpass_filter(image.astype('float'), window_sz, mask)
                kappa1, kappa2, var1, var2 = calc_cumulants(image_filtered, mask)
                
                if make_correction == True:
                    kappa2 *= window_sz / (window_sz - 1)
                    var2   *= (window_sz / (window_sz - 1))**2
                    
                np.save(join(folder, maskfolder, savefolder, mask_label+' kappa1.npy'), kappa1)
                np.save(join(folder, maskfolder, savefolder, mask_label+' kappa2.npy'), kappa2)
                np.save(join(folder, maskfolder, savefolder, mask_label+' var1.npy'), var1)
                np.save(join(folder, maskfolder, savefolder, mask_label+' var2.npy'), var2)